In [1]:
cd ..

/Users/kylemagida/git/blobs


In [2]:
import pysal as ps
import sys
import os
SHP_LNK = '/Users/kylemagida/git/blobs/tracts/CensusTractsTIGER2010.shp'
GEO_ID = "TRACTCE10"
root = os.getcwd()
WEIGHT_FILE = root + '/tracts/custom_weight_file.gal' # Use this file in creating blobs

Your version of PySAL is 185 days old.
There have likely been 1 new release(s).
Disable this check? [Y/n]n


In [10]:
# Run this to create a weight file for a custom set of shapefiles for use in blobs
rook_w = ps.rook_from_shapefile(SHP_LNK, idVariable=GEO_ID)
neighbors = rook_w.neighbors
island_dict = find_islands(neighbors)
print "Initial Islands:", len(island_dict)
print "Sizes:", [len(x) for x in island_dict.values()]
connect_islands(island_dict,rook_w)
'''
Islands are connected by running KNN on islands until they connect with the mainland, the 
mainland weights are unchanged.
'''
island_dict = find_islands(neighbors)
gal = ps.open(WEIGHT_FILE,'w')
gal.write(rook_w)
gal.close()

Initial Islands: 2
Sizes: [799, 2]


In [4]:
def get_connected_tracts(tract,neighbors,tract_dict):
    '''
    Recurses through all connected shapes to identify grouped areas
    '''
    dependent_tracts =  get_connected_tracts_helper(tract,[],neighbors,tract_dict)
    return list(dependent_tracts)

def get_connected_tracts_helper(tract,checked_tracts,neighbors,tract_dict):
    
    if len(checked_tracts) > 0 and tract in checked_tracts:
        return []
    
    if tract in tract_dict:
        return tract_dict[tract]
    
    dependent_tracts = set(neighbors[tract])
    checked_tracts.append(tract)

    for nearby_tract in neighbors[tract]:      
        second_level_tracts = get_connected_tracts_helper(nearby_tract,checked_tracts,neighbors,tract_dict)
        dependent_tracts |= set(second_level_tracts)
    tract_dict[tract] = list(dependent_tracts)
    
    return dependent_tracts

In [5]:
def find_islands(neighbors):
    '''
    Uses weights to identify all contiguous groups of tracts and returns the whole list
    '''
    island_dict = {}
    island_count = 0
    tract_dict = {}
    
    for tract in neighbors.keys():

        if tract_in_found_islands(island_dict,tract):
            continue
        else:
            dependent_tracts = get_connected_tracts(tract,neighbors,tract_dict)
            island_dict[island_count] = dependent_tracts
            island_count += 1
            
    return island_dict
        

def tract_in_found_islands(islands_dict,tract):
    '''
    Determines is a tract is already in an existing island
    '''
    for island_list in islands_dict.values():
        if tract in island_list:
            return True
    return False  

In [6]:
def connect_islands(island_dict,spatial_weights):
    '''
    Connects islands by having each tract in an island get weights with the KNN tracts when K mandates that at least one
    of the neighbors is not on the island
    '''
    mainland_size = 0
    mainland_num = None
    
    for island_num in island_dict:
        if len(island_dict[island_num]) > mainland_size:
            mainland_num = island_num
            mainland_size = len(island_dict[island_num])
    
    
    
    for island_num in island_dict:

        if island_num == mainland_num:
            continue
        else:
            joined = False
            island = island_dict[island_num]
            island_size = len(island)
            for knn in range(2,island_size+2):
                knn_weights = ps.knnW_from_shapefile(SHP_LNK,k=knn,idVariable=GEO_ID)
            #Add weights to new tracts that aren't on the island and to their pairs on other islands or the mainland
                for tract in island:
                    tracts_to_add = list(set(knn_weights.neighbors[tract]) - set(island))
                    if len(tracts_to_add) > 0:
                        joined = True
                        for new_tract in tracts_to_add:
                            # Add tract from off-island to tract neighbors (not vice-versa)
                            spatial_weights.neighbors[tract].append(new_tract)
                            spatial_weights.weights[tract].append(1)
                if joined:
                    break
